# SQL Part: Data Preparation and Variable Definition for Tableau Visualization

This analysis leverages Google BigQuery for efficient data processing and aggregation, with results then imported into a Jupyter Notebook to review the data preparation steps and have the data ready for visualization in Tableau.

In [35]:
import pandas as pd
from google.cloud import bigquery
from pandas_gbq import read_gbq 

project_id = 'sql-sandbox-1-450916'

   #### SQL CTE for Preparation of Visuals 1 and 3 (after project intro slides Siam Thai fight Manufactory)

In [36]:
CTE = """
WITH production_agg AS (
  SELECT
    p.factory_id,
    p.product_id,
    p.year,
    SUM(p.units_produced) AS total_units_produced,
    SUM(p.defective_units) AS total_defective_units
  FROM `sql-sandbox-1-450916.Siam_Thai_Fight_Manufactory.production_clean` p
  GROUP BY p.factory_id, p.product_id, p.year
),
sales_agg AS (
  SELECT
    s.product_id,
    EXTRACT(YEAR FROM s.sale_date) AS sale_year,
    SUM(s.units_sold) AS total_units_sold,
    SUM(s.revenue_usd) AS total_revenue,
    SUM(s.discount_applied) AS total_discount
  FROM `sql-sandbox-1-450916.Siam_Thai_Fight_Manufactory.sales_clean` s
  GROUP BY s.product_id, sale_year
)

SELECT
  p.factory_id,
  f.factory_name,
  f.location,
  f.Latitude,
  f.Longitude,
  f.num_employees,
  p.year,
  pr.product_id,
  pr.product_name,
  b.brand_name,
  pr.category,
  pr.macro_category,
  p.total_units_produced,
  p.total_defective_units,
  s.total_units_sold,
  s.total_revenue,
  s.total_discount,
  (p.total_units_produced * 1.0 / NULLIF(f.num_employees, 0)) AS units_per_employee,
  (pr.price_usd - pr.cost_usd) AS margin_per_unit,
  pr.discount_percent AS standard_discount
FROM production_agg p
JOIN `sql-sandbox-1-450916.Siam_Thai_Fight_Manufactory.factories_clean` f ON p.factory_id = f.factory_id
LEFT JOIN `sql-sandbox-1-450916.Siam_Thai_Fight_Manufactory.products_clean` pr ON p.product_id = pr.product_id
LEFT JOIN `sql-sandbox-1-450916.Siam_Thai_Fight_Manufactory.brands_clean` b ON pr.brand_id = b.brand_id
LEFT JOIN sales_agg s ON pr.product_id = s.product_id AND p.year = s.sale_year
ORDER BY p.factory_id, p.year, pr.product_id;
"""

df = read_gbq(CTE, project_id=project_id, dialect='standard')


df.head()

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,factory_id,factory_name,location,Latitude,Longitude,num_employees,year,product_id,product_name,brand_name,category,macro_category,total_units_produced,total_defective_units,total_units_sold,total_revenue,total_discount,units_per_employee,margin_per_unit,standard_discount
0,1,Bangkok HQ,Bangkok,13.7563,100.5018,190,2018,3,Gloves Top King,Top King,Gloves,Equipment,3109,85,170,28184.00,60,16.363158,35,20
1,1,Bangkok HQ,Bangkok,13.7563,100.5018,190,2018,6,Gloves Windy,Windy,Gloves,Equipment,2403,175,248,19758.00,100,12.647368,57,25
2,1,Bangkok HQ,Bangkok,13.7563,100.5018,190,2018,8,Gloves King Pro Boxing,King Pro Boxing,Gloves,Equipment,1114,48,174,31716.30,15,5.863158,24,15
3,1,Bangkok HQ,Bangkok,13.7563,100.5018,190,2018,16,Shin Guards Windy,Windy,Shin Guards,Equipment,901,12,228,32004.50,15,4.742105,29,5
4,1,Bangkok HQ,Bangkok,13.7563,100.5018,190,2018,29,Headgear Venum,Venum,Headgear,Equipment,3041,24,169,26754.45,60,16.005263,59,15


#### Second SQL Query for Preparation of Visuals 2 and 4 (following the project intro slides Siam thai Fight Manufactory visual)

In [37]:
Query_X_Visual="""
SELECT
  EXTRACT(YEAR FROM s.sale_date_mod) AS year,
  EXTRACT(MONTH FROM s.sale_date_mod) AS month,
  s.continent,
  s.country,
  p.macro_category,
  p.category,
  b.brand_name,
  s.channel,
  
  SUM(s.units_sold) AS total_units_sold,
  SUM(s.revenue_usd) AS total_revenue,
  SUM(s.units_sold * p.margin_usd_mod) AS total_margin_usd,
  SUM(s.loss_due_to_discount) AS total_loss_due_to_discount,
  COUNT(DISTINCT s.sale_id) AS total_transactions
  
FROM `sql-sandbox-1-450916.Siam_Thai_Fight_Manufactory.sales_clean` s

JOIN `sql-sandbox-1-450916.Siam_Thai_Fight_Manufactory.products_clean` p
  ON s.product_id = p.product_id

JOIN `sql-sandbox-1-450916.Siam_Thai_Fight_Manufactory.brands_clean` b
  ON p.brand_id = b.brand_id

GROUP BY
  year,
  month,
  s.continent,
  s.country,
  p.macro_category,
  p.category,
  b.brand_name,
  s.channel

ORDER BY
  year,
  month,
  s.continent,
  s.country,
  p.macro_category,
  p.category,
  b.brand_name,
  s.channel
"""

df1 = read_gbq(Query_X_Visual, project_id=project_id, dialect='standard')


df1.head()

c:\Users\Sker\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:2660: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,year,month,continent,country,macro_category,category,brand_name,channel,total_units_sold,total_revenue,total_margin_usd,total_loss_due_to_discount,total_transactions
0,2018,1,Africa,South Africa,Accessories,Ankle Supports,Booster Fight Gear,Retail,27,539.0,334.152190,0.0,1
1,2018,1,Africa,South Africa,Accessories,Ankle Supports,Danger Equipment,Online,5,200.0,23.678762,0.0,1
2,2018,1,Africa,South Africa,Accessories,Hand Wraps,Booster Fight Gear,Retail,7,420.0,26.611123,0.0,1
3,2018,1,Africa,South Africa,Accessories,Hand Wraps,Fairtex,Retail,24,572.0,256.054407,0.0,1
4,2018,1,Africa,South Africa,Accessories,Hand Wraps,Venum,Distributor,1,93.0,11.562932,0.0,1
